In [23]:
from sklearn.cluster import KMeans
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [24]:
cluster_info = np.loadtxt(open("test_files/test.csv", "r"), delimiter=",", skiprows=1)
cluster_info

array([[25.  ,  2.  ,  4.25,  3.  ],
       [21.  ,  3.  ,  1.  ,  2.  ],
       [17.  ,  4.  ,  3.25, -3.  ],
       [22.  ,  5.  ,  6.75, -4.  ]])

In [7]:
k_size = int(len(cluster_info)/2)
print(k_size)
kmeans = KMeans(n_clusters=k_size).fit(cluster_info)

2


NameError: name 'KMeans' is not defined

In [8]:
kmeans.labels_

NameError: name 'kmeans' is not defined

In [13]:
#kmeans.predict(fill in with our own 7 unit vector data), will classify it into cluster

In [17]:
class Node:
    def __init__(self, node_num):
        self.node_num = node_num
        self.s_exp = []
        self.cpt = {}
        self.count = 0
    
    def add_exp(self, s):
        self.s_exp.append(s)

In [18]:
s_exp_dict = {}

s_exp_labels = np.loadtxt(open("test_files/test_exp.csv", "r"), dtype="str", delimiter=",", skiprows=1)

for expression in s_exp_labels:
    #s-expression, song-id (which song), song-index (position in song)
    s_exp_dict[expression[0]] = [expression[1], expression[2], expression[3]]

print(s_exp_dict)

{'0': ['asdf', '1', '0'], '1': ['fghh', '1', '1'], '2': ['dfgj', '2', '0'], '3': ['sdfw', '2', '1']}


In [19]:
node_objects = []

for cluster_num in range(k_size):
    node_objects.append(Node(cluster_num))
    
for i in range(0,len(kmeans.labels_)):
    cluster_num = kmeans.labels_[i]
    node_objects[cluster_num].add_exp(s_exp_dict[str(i)]) #adding s-expression, song-id (which song), song-index (position in song) to the node object

NameError: name 'kmeans' is not defined

In [22]:
#create cpt
for outer_node in node_objects:
    for inner_node in node_objects:
        outer_node.cpt[inner_node.node_num] = 0
        if (outer_node != inner_node):
            for s_expression_outer in outer_node.s_exp:
                for s_expression_inner in inner_node.s_exp:
                    if s_expression_outer[1] == s_expression_inner[1] and s_expression_inner[2] - s_expression_outer[2] == 1:
                        outer_node.count += 1
                        outer_node.cpt[inner_node.node_num] += 1
    outer_node.cpt = {k:v/outer_node.count for k, v in outer_node.cpt.items() if outer_node.count != 0}